# Train GTAV Self-Driving Model

In [ ]:
# Import relevant packages
import random
import ntpath

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as npimg  
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa

import torch
import torch.nn as nn
from torch.optim import Adam

In [ ]:
# Establish fixed values
